# ML2CPP

## Preparing the dataset

In [1]:
from sklearn import datasets
import numpy as np
import pandas as pd

boston = datasets.load_boston()

def populate_table(tablename, feature_names):
    X = boston.data  
    y = boston.target
    N = X.shape[0]
    y = y.reshape(N,1)
    k = np.arange(N).reshape(N, 1)
    k_X_y = np.concatenate((k, X, y) , axis=1)
    lTable=pd.DataFrame(k_X_y)
    # print(lTable.head())
    lTable.columns = ['idx'] + feature_names + ['TGT'];
    lTable['TGT'] = lTable['TGT'].apply(int)
    lTable['idx'] = lTable['idx'].apply(int)
    lTable.to_csv(tablename , float_format='%.14g')



In [2]:
metadata = {"primary_key" : "KEY",
            "features" : list(boston.feature_names),
            "targets" : ["TGT"],
            "table" : "iris"}

In [3]:
populate_table("/tmp/boston.csv" , metadata["features"])


In [4]:
df = pd.read_csv("/tmp/boston.csv")
df.sample(12, random_state=1960)

,Unnamed: 0,idx,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,TGT
230,230,230,0.53700,0.0,6.20,0,0.504,5.981,68.1,3.6715,8,307,17.4,378.35,11.65,24
112,112,112,0.12329,0.0,10.01,0,0.547,5.913,92.9,2.3534,6,432,17.8,394.95,16.21,18
125,125,125,0.16902,0.0,25.65,0,0.581,5.986,88.4,1.9929,2,188,19.1,385.02,14.81,21
9,9,9,0.17004,12.5,7.87,0,0.524,6.004,85.9,6.5921,5,311,15.2,386.71,17.10,18
213,213,213,0.14052,0.0,10.59,0,0.489,6.375,32.3,3.9454,4,277,18.6,385.81,9.38,28
109,109,109,0.26363,0.0,8.56,0,0.520,6.229,91.2,2.5451,5,384,20.9,391.23,15.55,19
127,127,127,0.25915,0.0,21.89,0,0.624,5.693,96.0,1.7883,4,437,21.2,392.11,17.19,16
244,244,244,0.20608,22.0,5.86,0,0.431,5.593,76.5,7.9549,7,330,19.1,372.49,12.50,17
406,406,406,20.71620,0.0,18.10,0,0.659,4.138,100.0,1.1781,24,666,20.2,370.22,23.34,11
490,490,490,0.20746,0.0,27.74,0,0.609,5.093,98.0,1.8226,4,711,20.1,318.43,29.68,8


## Training a Model

In [5]:

from sklearn.decomposition import TruncatedSVD

clf = TruncatedSVD(n_components = 10)

clf.fit(df[metadata['features']].values, df[metadata['targets']].values)


TruncatedSVD(n_components=10)

## Deploying the Model

In [6]:

def generate_cpp_for_model(model):
    import pickle, json, requests, base64
    b64_data = base64.b64encode(pickle.dumps(model)).decode('utf-8')
    # send the model th the web service
    json_data={"Name":"model_cpp_sample", 
               "PickleData":b64_data , 
               "SQLDialect":"CPP",
               "FeatureNames" : metadata['features']}
    r = requests.post("https://sklearn2sql.herokuapp.com/model", json=json_data)
    content = r.json()
    lCPP = content["model"]["SQLGenrationResult"][0]["SQL"]
    # print(lCPP);
    return lCPP


lCPPCode = generate_cpp_for_model(clf);


In [7]:
print(lCPPCode)

namespace  {

	std::vector<std::string> get_input_names(){
		std::vector<std::string> lFeatures = { "Feature_0", "Feature_1", "Feature_2", "Feature_3", "Feature_4", "Feature_5", "Feature_6", "Feature_7", "Feature_8", "Feature_9", "Feature_10", "Feature_11", "Feature_12" };

		return lFeatures;
	}

	std::vector<std::string> get_output_names(){
		std::vector<std::string> lOutputs = { "trunc_svd_1", "trunc_svd_2", "trunc_svd_3", "trunc_svd_4", "trunc_svd_5", "trunc_svd_6", "trunc_svd_7", "trunc_svd_8", "trunc_svd_9", "trunc_svd_10" };

		return lOutputs;
	}

	tTable compute_features(std::any Feature_0, std::any Feature_1, std::any Feature_2, std::any Feature_3, std::any Feature_4, std::any Feature_5, std::any Feature_6, std::any Feature_7, std::any Feature_8, std::any Feature_9, std::any Feature_10, std::any Feature_11, std::any Feature_12) {

		tTable lTable;

		lTable["trunc_svd_1"] = { Feature_0 * 0.007828890925998899 + Feature_1 * 0.017421275249209445 + Feature_2 * 0.02111326865873322

In [8]:
    def write_text_to_file(iCPPCode, oCPPFile):          
        with open(oCPPFile, "w") as text_file:
            text_file.write(iCPPCode)

    def add_cpp_main_function(iCPPCode, iCSVFile):
        lCPPCode = "#include \"Generic.i\"\n\n"
        lCPPCode = lCPPCode + iCPPCode
        lCPPCode = lCPPCode + "\tint main() {\n"
        lCPPCode = lCPPCode + "\t\tscore_csv_file(\"" + iCSVFile +"\");\n"
        lCPPCode = lCPPCode + "\treturn 0;\n}\n"
        return lCPPCode

    def compile_cpp_code_as_executable(iName):
        import subprocess
        lCommand = ["g++", "-Wall", "-Wno-unused-function", "-std=c++17" , "-g" ,  "-o", iName + ".exe",  iName + ".cpp"]
        print("EXECUTING" , "'" + " ".join(lCommand) + "'")
        result = subprocess.check_output(lCommand)
        # print(result)

    def execute_cpp_model(iName, iCSVFile):
        import subprocess
        result2 = subprocess.check_output([iName + ".exe",  iCSVFile])
        result2 = result2.decode()
        print(result2[:100])
        print(result2[-100:])
        return result2
        
    def execute_cpp_code(iCPPCode, iCSVFile):
        lName = "/tmp/sklearn2sql_cpp_" + str(id(clf));
        lCPPCode = add_cpp_main_function(iCPPCode, iCSVFile)
        write_text_to_file(lCPPCode, lName + ".cpp")
        compile_cpp_code_as_executable(lName)
        result = execute_cpp_model(lName, iCSVFile)
        write_text_to_file(str(result), lName + ".out")
        return lName + ".out"


In [9]:
populate_table("/tmp/boston2.csv" , ["Feature_" + str(i) for i,x in enumerate(metadata["features"])])
lCPPOutput = execute_cpp_code(lCPPCode , "/tmp/boston2.csv")
cpp_output = pd.read_csv(lCPPOutput)

EXECUTING 'g++ -Wall -Wno-unused-function -std=c++17 -g -o /tmp/sklearn2sql_cpp_140420607529120.exe /tmp/sklearn2sql_cpp_140420607529120.cpp'
idx,trunc_svd_1,trunc_svd_2,trunc_svd_3,trunc_svd_4,trunc_svd_5,trunc_svd_6,trunc_svd_7,trunc_svd_8,
70841880312,-2.02164609517863,3.43011784189083,3.87838556433779,-2.15006030289988,-1.64330138596451



In [10]:
cpp_output.sample(12, random_state=1960)

,idx,trunc_svd_1,trunc_svd_2,trunc_svd_3,trunc_svd_4,trunc_svd_5,trunc_svd_6,trunc_svd_7,trunc_svd_8,trunc_svd_9,trunc_svd_10
230,230,481.857584,-100.515072,-17.189539,-4.545158,0.389338,-1.138891,-4.239366,-0.554642,-0.392278,0.071637
112,112,591.759033,-34.760415,-23.485297,2.754648,-3.941234,0.239090,-2.004357,0.712809,5.454576,-0.656369
125,125,397.248975,-179.105064,-48.024188,13.415853,1.661355,7.297125,12.111536,-4.002741,-9.732061,0.060612
9,9,492.629748,-104.761754,-23.468610,15.565032,0.053105,2.504255,-3.263259,-1.882112,2.570127,3.414451
213,213,458.927040,-126.144523,7.224366,-25.039124,0.796584,4.325337,2.056346,1.676666,-2.723674,-0.526110
109,109,552.283264,-61.941828,-27.311110,4.673897,-2.610024,0.586599,-3.243749,2.958736,2.460706,-1.636839
127,127,594.571160,-29.201065,-26.614932,4.605517,-4.997113,5.275849,7.593097,1.167700,0.076654,-1.170186
244,244,497.431642,-82.755539,-7.881929,17.379951,-1.013617,-0.484153,-4.419296,1.106527,-0.769997,2.816734
406,406,758.465302,132.070343,-5.303445,-3.459492,12.091105,-0.287550,-0.543973,-4.213795,1.745197,-0.732173
490,490,760.462113,199.201601,-0.745117,-1.920602,-12.885455,16.266200,8.801234,4.191429,10.410400,-0.513777


In [11]:
skl_outputs = pd.DataFrame()
X = df[metadata['features']].values
skl_output_key = pd.DataFrame(list(range(X.shape[0])), columns=['idx']);

skl_output_transform = pd.DataFrame(clf.transform(X), columns=cpp_output.columns[1:]);
skl_output = pd.concat([skl_output_key, skl_output_transform] , axis=1)
skl_output.sample(12, random_state=1960)


,idx,trunc_svd_1,trunc_svd_2,trunc_svd_3,trunc_svd_4,trunc_svd_5,trunc_svd_6,trunc_svd_7,trunc_svd_8,trunc_svd_9,trunc_svd_10
230,230,481.857584,-100.515072,-17.189539,-4.545158,0.389338,-1.138891,-4.239366,-0.554642,-0.392278,0.071637
112,112,591.759033,-34.760415,-23.485297,2.754648,-3.941234,0.239090,-2.004357,0.712809,5.454576,-0.656369
125,125,397.248975,-179.105064,-48.024188,13.415853,1.661355,7.297125,12.111536,-4.002741,-9.732061,0.060612
9,9,492.629748,-104.761754,-23.468610,15.565032,0.053105,2.504255,-3.263259,-1.882112,2.570127,3.414451
213,213,458.927040,-126.144523,7.224366,-25.039124,0.796584,4.325337,2.056346,1.676666,-2.723674,-0.526110
109,109,552.283264,-61.941828,-27.311110,4.673897,-2.610024,0.586599,-3.243749,2.958736,2.460706,-1.636839
127,127,594.571160,-29.201065,-26.614932,4.605517,-4.997113,5.275849,7.593097,1.167700,0.076654,-1.170186
244,244,497.431642,-82.755539,-7.881929,17.379951,-1.013617,-0.484153,-4.419296,1.106527,-0.769997,2.816734
406,406,758.465302,132.070343,-5.303445,-3.459492,12.091105,-0.287550,-0.543973,-4.213795,1.745197,-0.732173
490,490,760.462113,199.201601,-0.745117,-1.920602,-12.885455,16.266200,8.801234,4.191429,10.410400,-0.513777


In [12]:
cpp_skl_join = skl_output.join(cpp_output , how='left', on='idx', lsuffix='_skl', rsuffix='_cpp')

In [13]:
cpp_skl_join.sample(12, random_state=1960)

,idx_skl,trunc_svd_1_skl,trunc_svd_2_skl,trunc_svd_3_skl,trunc_svd_4_skl,trunc_svd_5_skl,trunc_svd_6_skl,trunc_svd_7_skl,trunc_svd_8_skl,trunc_svd_9_skl,...,trunc_svd_1_cpp,trunc_svd_2_cpp,trunc_svd_3_cpp,trunc_svd_4_cpp,trunc_svd_5_cpp,trunc_svd_6_cpp,trunc_svd_7_cpp,trunc_svd_8_cpp,trunc_svd_9_cpp,trunc_svd_10_cpp
230,230,481.857584,-100.515072,-17.189539,-4.545158,0.389338,-1.138891,-4.239366,-0.554642,-0.392278,...,481.857584,-100.515072,-17.189539,-4.545158,0.389338,-1.138891,-4.239366,-0.554642,-0.392278,0.071637
112,112,591.759033,-34.760415,-23.485297,2.754648,-3.941234,0.239090,-2.004357,0.712809,5.454576,...,591.759033,-34.760415,-23.485297,2.754648,-3.941234,0.239090,-2.004357,0.712809,5.454576,-0.656369
125,125,397.248975,-179.105064,-48.024188,13.415853,1.661355,7.297125,12.111536,-4.002741,-9.732061,...,397.248975,-179.105064,-48.024188,13.415853,1.661355,7.297125,12.111536,-4.002741,-9.732061,0.060612
9,9,492.629748,-104.761754,-23.468610,15.565032,0.053105,2.504255,-3.263259,-1.882112,2.570127,...,492.629748,-104.761754,-23.468610,15.565032,0.053105,2.504255,-3.263259,-1.882112,2.570127,3.414451
213,213,458.927040,-126.144523,7.224366,-25.039124,0.796584,4.325337,2.056346,1.676666,-2.723674,...,458.927040,-126.144523,7.224366,-25.039124,0.796584,4.325337,2.056346,1.676666,-2.723674,-0.526110
109,109,552.283264,-61.941828,-27.311110,4.673897,-2.610024,0.586599,-3.243749,2.958736,2.460706,...,552.283264,-61.941828,-27.311110,4.673897,-2.610024,0.586599,-3.243749,2.958736,2.460706,-1.636839
127,127,594.571160,-29.201065,-26.614932,4.605517,-4.997113,5.275849,7.593097,1.167700,0.076654,...,594.571160,-29.201065,-26.614932,4.605517,-4.997113,5.275849,7.593097,1.167700,0.076654,-1.170186
244,244,497.431642,-82.755539,-7.881929,17.379951,-1.013617,-0.484153,-4.419296,1.106527,-0.769997,...,497.431642,-82.755539,-7.881929,17.379951,-1.013617,-0.484153,-4.419296,1.106527,-0.769997,2.816734
406,406,758.465302,132.070343,-5.303445,-3.459492,12.091105,-0.287550,-0.543973,-4.213795,1.745197,...,758.465302,132.070343,-5.303445,-3.459492,12.091105,-0.287550,-0.543973,-4.213795,1.745197,-0.732173
490,490,760.462113,199.201601,-0.745117,-1.920602,-12.885455,16.266200,8.801234,4.191429,10.410400,...,760.462113,199.201601,-0.745117,-1.920602,-12.885455,16.266200,8.801234,4.191429,10.410400,-0.513777


In [14]:
for col in cpp_output.columns:
    lDiff = cpp_skl_join[col + "_skl"] - cpp_skl_join[col + "_cpp"]
    print(lDiff.describe())
    

count    506.0
mean       0.0
std        0.0
min        0.0
25%        0.0
50%        0.0
75%        0.0
max        0.0
dtype: float64
count    5.060000e+02
mean     5.279922e-15
std      6.199256e-14
min     -1.136868e-13
25%      0.000000e+00
50%      0.000000e+00
75%      5.684342e-14
max      2.273737e-13
dtype: float64
count    5.060000e+02
mean    -4.691021e-16
std      1.639396e-14
min     -1.136868e-13
25%      0.000000e+00
50%      0.000000e+00
75%      0.000000e+00
max      1.136868e-13
dtype: float64
count    5.060000e+02
mean    -2.235942e-16
std      3.786678e-15
min     -1.421085e-14
25%     -3.393119e-15
50%      0.000000e+00
75%      2.400857e-15
max      1.421085e-14
dtype: float64
count    5.060000e+02
mean    -1.671917e-16
std      3.294445e-15
min     -7.105427e-15
25%     -3.552714e-15
50%      0.000000e+00
75%      2.664535e-15
max      7.105427e-15
dtype: float64
count    5.060000e+02
mean     7.037543e-17
std      3.014455e-15
min     -1.421085e-14
25%     -2.22